## init install

In [ ]:
!pip install django
!pip install line-bot-sdk
!pip install beautifulsoup4
!pip install requests
!pip install transitions

## load google drive and creat ngrok file

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh

!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

## ngrok

In [ ]:
!/ngrok authtoken 2JARutcFOzH6UTqWcwl5byNtTEh_3xyGd9rVQuHe9zTVNcyXt
!pip install flask_ngrok

## linebot token and access

In [4]:
access_token = '093OyYqGonM6q3yRXZcRg5WtTP0m3P7ia1WuwgcP91lYqaHb6Bx1sbW2G5YayruFh+hJyTOjdWECR0zFDBgRyieQU/vLUFDqHs2Z+lyKfuj7ZSRycGgxV8wSDZxlWgR/wSvqKSucv80Be0TGqUFkRwdB04t89/1O/w1cDnyilFU='
secret = 'bf79fc94041fac8f854c411bc57ed460'

## fsm function

In [5]:
from zmq.constants import TCP_KEEPALIVE_CNT
from transitions.extensions import GraphMachine

class TocMachine(GraphMachine):
    def __init__(self, **machine_configs):
        self.machine = GraphMachine(model=self, **machine_configs)

    def is_going_to_reserve(self, event):
        text = event['message']['text']
        return text.lower() == "reserve"

    def is_going_to_cancel(self, event):
        text = event['message']['text']
        return text.lower() == "cancel"

    def is_going_to_help(self, event):
        text = event['message']['text']
        return text.lower() == "help"

    def on_enter_reserve(self, event):
        print("I'm entering reserve")
        line_bot_api = LineBotApi(access_token)
        reply_token = event['replyToken']
        line_bot_api.reply_message(reply_token, TextSendMessage(text="預約成功"))
        self.go_back()

    def on_enter_cancel(self, event):
        print("I'm entering cancel")
        line_bot_api = LineBotApi(access_token)
        reply_token = event['replyToken']
        line_bot_api.reply_message(reply_token, TextSendMessage("取消成功"))
        self.go_back()

    def on_enter_help(self, event):
        print("I'm entering help")
        txt = "可用指令:\n"+"\t"+"reserve"+"\t"+" 預約\n"+"\t"+"cancel"+"\t"+" 取消預約\n"+"\t"+"help"+"\t"+" 指令查詢\n"
        line_bot_api = LineBotApi(access_token)
        reply_token = event['replyToken']
        line_bot_api.reply_message(reply_token, TextSendMessage(txt))
        self.go_back()

## app function

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, send_file

from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

import json

states=["user", "reserve", "cancel", "help"]

machine = TocMachine(
    states=["user", "reserve", "cancel", "help"],
    transitions=[
        {
            "trigger": "advance",
            "source": "user",
            "dest": "reserve",
            "conditions": "is_going_to_reserve",
        },
        {
            "trigger": "advance",
            "source": "user",
            "dest": "cancel",
            "conditions": "is_going_to_cancel",
        },
        {
            "trigger": "advance",
            "source": "user",
            "dest": "help",
            "conditions": "is_going_to_help",
        },
        {"trigger": "go_back", "source": ["reserve", "cancel", "help"], "dest": "user"},
    ],
    initial="user",
    auto_transitions=False,
    show_conditions=True,
)

app = Flask(__name__)

@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)
    json_data = json.loads(body)                    
    try:                         
        line_bot_api = LineBotApi(access_token)              
        handler = WebhookHandler(secret)                     
        signature = request.headers['X-Line-Signature']      
        handler.handle(body, signature)
        tk = json_data['events'][0]['replyToken']
        
        if(json_data['events'][0]["type"] != 'message'):
            line_bot_api.reply_message(tk, TextSendMessage("請輸入有效指令，可使用help查詢")) 
        elif(json_data["events"][0]["message"]["type"] != 'text'):
            line_bot_api.reply_message(tk, TextSendMessage("請輸入有效指令，可使用help查詢"))
        elif(json_data["events"][0]["message"]["text"] not in states):
            line_bot_api.reply_message(tk, TextSendMessage("請輸入有效指令，可使用help查詢"))  
        else:                    
            machine.advance(json_data["events"][0])
        print(f"\nFSM STATE: {machine.state}")
        print(f"REQUEST BODY: \n{body}")
                                    
    except:
        print() 

      

    return 'OK'   

@app.route("/show-fsm", methods=["GET"])
def show_fsm():
    machine.get_graph().draw("fsm.png", prog="dot", format="png")
    return send_file("fsm.png", mimetype="image/png")

if __name__ == "__main__":
  run_with_ngrok(app)      
  app.run()

## test

In [51]:
from zmq.constants import TCP_KEEPALIVE_CNT
from transitions.extensions import GraphMachine

class TocMachine(GraphMachine):
    def __init__(self, **machine_configs):
        self.machine = GraphMachine(model=self, **machine_configs)
        
    def is_going_to_reserve_double_check(self, event):
        text = event['message']['text']
        line_bot_api = LineBotApi(access_token)
        reply_token = event['replyToken']
        if text.lower() not in ["yes", "no"]:
          txt = "請輸入(yes/no)"
          line_bot_api.reply_message(reply_token, TextSendMessage(txt))
        return text.lower() in ["yes", "no"]

    def is_going_to_cancel_double_check(self, event):
        text = event['message']['text']
        if text.lower() not in ["yes", "no"]:
          txt = "請輸入(yes/no)"
          line_bot_api = LineBotApi(access_token)
          reply_token = event['replyToken']
          line_bot_api.reply_message(reply_token, TextSendMessage(txt))
        return text.lower() in ["yes", "no"]

    def is_going_to_reserve(self, event):
        text = event['message']['text']
        reply_token = event['replyToken']
        line_bot_api = LineBotApi(access_token)
        if text.lower() == "reserve":
          txt = "請確認預約(yes/no)"
        elif text.lower() == "cancel":
          txt = "尚未預約"
        else:
          txt = "指令錯誤，可輸入help查詢"
        line_bot_api.reply_message(reply_token, TextSendMessage(txt))
        return text.lower() == "reserve"

    def is_going_to_cancel(self, event):
        text = event['message']['text']
        reply_token = event['replyToken']
        line_bot_api = LineBotApi(access_token)
        if text.lower() == "cancel":
          txt = "請確認預約(yes/no)"
        elif text.lower() == "reserve":
          txt = "尚未預約"
        else:
          txt = "指令錯誤，可輸入help查詢"
        line_bot_api.reply_message(reply_token, TextSendMessage(txt))
        return text.lower() == "cancel"

    def is_going_to_help(self, event):
        text = event['message']['text']
        if text.lower() == "help":
          txt = "可使用指令:\n\t"+"reserve\t: 預約\n\t"+"cancel\t: 取消預約\n\t"+"help\t: 指令列"
          line_bot_api = LineBotApi(access_token)
          reply_token = event['replyToken']
          line_bot_api.reply_message(reply_token, TextSendMessage(txt))
        return False

    def on_enter_reserve(self, event):
        print("I'm entering reserve")

    def on_enter_cancel(self, event):
        print("I'm entering cancel")

    def on_enter_double_check_reserve(self, event):
        print("I'm entering double check")
        text = event['message']['text']
        if text.lower() == "no":
            self.go_to_user()
        else:
            self.go_to_user_reserve()
        


    def on_enter_double_check_cancel(self, event):
        print("I'm entering double check")
        text = event['message']['text']
        if text.lower() == "yes":
            self.go_to_user()
        else:
            self.go_to_user_reserve()

In [52]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, send_file

from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

import json

states=["yes", "no", "reserve", "cancel", "help"]

machine = TocMachine(
    states=["user", "user_reserve", "reserve", "cancel", "help", "double_check_reserve", "double_check_cancel"],
    transitions=[
        {
            "trigger": "advance",
            "source": "user",
            "dest": "reserve",
            "conditions": "is_going_to_reserve",
        },
        {
            "trigger": "advance",
            "source": "user",
            "dest": "cancel",
            "conditions": "is_going_to_cancel",
        },
        {
            "trigger": "advance",
            "source": ["user", "user_reserve", "reserve", "cancel", "double_check_reserve", "double_check_cancel"],
            "dest": "help",
            "conditions": "is_going_to_help",
        },
        {
            "trigger": "advance",
            "source": "reserve",
            "dest": "double_check_reserve",
            "conditions": "is_going_to_double_check_reserve",
        },
        {
            "trigger": "advance",
            "source": "cancel",
            "dest": "double_check_cancel",
            "conditions": "is_going_to_double_check_cancel",
        },
        {"trigger": "go_to_user", "source": ["double_check_reserve", "double_check_cancel"], "dest": "user"},
        {"trigger": "go_to_user_reserve", "source": ["double_check_reserve", "double_check_cancel"], "dest": "user_reserve"},
        {"trigger": "go_back", "source": ["user_reserve", "reserve", "cancel", "double_check_reserve", "double_check_cancel"], "dest": "user_reserve"},
    ],
    initial="user",
    auto_transitions=False,
    show_conditions=True,
)

app = Flask(__name__)

@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)
    json_data = json.loads(body)                    
    try:                         
        line_bot_api = LineBotApi(access_token)              
        handler = WebhookHandler(secret)                     
        signature = request.headers['X-Line-Signature']      
        handler.handle(body, signature)
        tk = json_data['events'][0]['replyToken']
        
        if(json_data['events'][0]["type"] != 'message'):
            line_bot_api.reply_message(tk, TextSendMessage("請輸入有效指令，可以使用help查詢")) 
        elif(json_data["events"][0]["message"]["type"] != 'text'):
            line_bot_api.reply_message(tk, TextSendMessage("請輸入有效指令，可以使用help查詢"))
        elif(json_data["events"][0]["message"]["text"] not in states):
            line_bot_api.reply_message(tk, TextSendMessage("請輸入有效指令，可以使用help查詢"))  
        else:                    
            machine.advance(json_data["events"][0])
        print(f"\nFSM STATE: {machine.state}")
        print(f"REQUEST BODY: \n{body}")
                                    
    except:
        print() 

      

    return 'OK'   

@app.route("/show-fsm", methods=["GET"])
def show_fsm():
    machine.get_graph().draw("fsm.png", prog="dot", format="png")
    return send_file("fsm.png", mimetype="image/png")

if __name__ == "__main__":
  #show_fsm()
  run_with_ngrok(app)      
  app.run()

 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1876-34-86-146-171.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Dec/2022 12:49:10] "POST / HTTP/1.1" 200 -


INFO:werkzeug:127.0.0.1 - - [23/Dec/2022 12:49:17] "POST / HTTP/1.1" 200 -


I'm entering reserve

FSM STATE: reserve
REQUEST BODY: 
{"destination":"Ua07c9542212fcff4aef26ed06613a24c","events":[{"type":"message","message":{"type":"text","id":"17345996630571","text":"reserve"},"webhookEventId":"01GMZGFNY3NWDTJ92Q62Y1MGEP","deliveryContext":{"isRedelivery":false},"timestamp":1671799756230,"source":{"type":"user","userId":"Ud3c585c73761c3bff026fe8fb8806743"},"replyToken":"3c9f7352875744ad98bae698892f1334","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [23/Dec/2022 12:49:21] "POST / HTTP/1.1" 200 -


INFO:werkzeug:127.0.0.1 - - [23/Dec/2022 12:49:46] "POST / HTTP/1.1" 200 -


INFO:werkzeug:127.0.0.1 - - [23/Dec/2022 12:50:15] "POST / HTTP/1.1" 200 -
